# [Julia ASTs](https://docs.julialang.org/en/v1.0/devdocs/ast/)

Julia has two representations of code.

1. The surface syntax AST returned by the parser (e.g. the `Meta.parse` function), and manipulated by macros.
    - It is a structured representation of code as it is written, constructed by `julia-parser.scm` from a character stream.
1. The lowered form constructed by `julia-syntax.scm`, or IR (intermediate representation), which is used by _**type inference**_ and _**code generation**_.
    - In the lowered form there are fewer types of nodes:
        - all macros are expanded
        - all control flow is converted to explicit branches and sequences of statements

>_**The lowered form is more important to the compiler, but less obvious to the human, since it results from a significant rearrangement of the input syntax.**_

## Data types exist in lowered form:

**[TODO] This part is too abstract to understand. I will revisit it again after I have some concrete code snippets.**

- `Expr`, Also see this documentation for reference: _[program representation](https://docs.julialang.org/en/v1/manual/metaprogramming/#Program-representation-1)_.

    - _The below sentence is in the documentation, but I cannot understand very well._
        > While almost every part of a surface AST is represented by an Expr, the IR uses only a limited number of Exprs, mostly for calls, conditional branches (gotoifnot), and returns.

In [9]:
prog = :(1 + 1)

@show typeof(prog)  # Expr is a type in lowered form AST.

@show typeof(prog.head)
@show typeof(prog.args);

typeof(prog) = Expr
typeof(prog.head) = Symbol
typeof(prog.args) = Array{Any,1}


# [Julia SSA-form IR](https://docs.julialang.org/en/v1.0/devdocs/ssair/)

>_Beginning in Julia 0.7, **parts** of the compiler use a new SSA-form intermediate representation._

[TODO] _**what parts?**_

- Historically, the compiler used to directly generate LLVM IR, from a lowered form of the Julia AST.
    - This IR had most syntactic abstractions removed, but still looked a lot like an abstract syntax tree.

- In order to facilitate optimizations, SSA values were introduced to the lowered form IR and the IR was linearized
    - i.e. a form where function arguments may only be SSA values or constants.

## Four new IR nodes in SSA format

SSA format introduces four IR nodes:

- Phi nodes, Pi nodes
- PhiC nodes and Upsilon nodes: only used for exception handling

### Phi node and Pi note

Pi and Phi nodes in the IR are necessary for [back-edges](https://stackoverflow.com/questions/44494426/back-edges-in-a-graph/44494705) and re-merging of conditional control flow.

- _**Why Phi and Pi node**_

    _**Non-ssa values (slots, see [this documentation](https://docs.julialang.org/en/v1.0/devdocs/ast/#Lowered-form-1) for the explanatioin of slots)**_ in the IR negate much of the usefulfulness of the SSA form representation to perform middle end optimization.